# Azure AI Foundry + Semantic Kernel: Code Interpreter Tutorial

A clean, educational example showing how to:
1. Create an Azure AI Foundry agent with code interpreter
2. Wrap it with Semantic Kernel for conversation management  
3. Generate files with the agent
4. Retrieve and save files locally
5. Clean up resources properly

## Prerequisites
- Azure AI Foundry project with agent service enabled
- Environment variables: `PROJECT_ENDPOINT`, `MODEL_DEPLOYMENT_NAME`
- Libraries: `semantic-kernel>=1.37.0`, `azure-ai-agents>=1.2.0b4`

## 1. Import Libraries & Setup

In [8]:
import os
import nest_asyncio
from pathlib import Path
from dotenv import load_dotenv

# Azure credentials
from azure.identity import DefaultAzureCredential

# Azure AI Foundry SDK
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import CodeInterpreterTool

# Semantic Kernel
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread
from azure.ai.projects.aio import AIProjectClient

# Setup
nest_asyncio.apply()
load_dotenv()

# Create downloads directory
downloads_dir = Path.cwd() / "azure_ai_files" / "downloads"
downloads_dir.mkdir(parents=True, exist_ok=True)

print("✅ Setup complete!")

✅ Setup complete!


## 2. Create Azure AI Foundry Agent

In [2]:
# Create credential and foundry client
credential = DefaultAzureCredential()
foundry_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=credential
)

# Create agent with code interpreter
code_interpreter = CodeInterpreterTool()
agent_definition = foundry_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="ChartAgent",
    instructions="Create data visualizations and save as files.",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources
)

print(f"✅ Agent created: {agent_definition.id}")

✅ Agent created: asst_OlKWEGQLb5JU7qcPH2fBtPfo


## 3. Create Semantic Kernel Wrapper & Thread

In [3]:
# Create Semantic Kernel client and agent wrapper
sk_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=credential
)

agent = AzureAIAgent(client=sk_client, definition=agent_definition)
thread = AzureAIAgentThread(client=sk_client)

print("✅ Semantic Kernel agent and thread ready")

✅ Semantic Kernel agent and thread ready


## 4. Chat with Agent (Generate Files)

In [4]:
# Ask agent to create a chart
message = "Create a colorful bar chart showing monthly data (Jan=31, Feb=28, Mar=31, Apr=30, May=31, Jun=30) and save as a file."

async for response in agent.invoke(messages=message, thread=thread):
    print(response.content)

print("\n✅ Agent response complete")

import matplotlib.pyplot as plt

# Data for the bar chart
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
values = [31, 28, 31, 30, 31, 30]
colors = ['#FF6F61', '#6B5B95', '#88B04B', '#F7CAC9', '#92A8D1', '#FFD700'] # Distinct and colorful

# Create the bar chart
plt.figure(figsize=(8, 6))
bars = plt.bar(months, values, color=colors)
plt.xlabel('Month')
plt.ylabel('Value')
plt.title('Monthly Data')
plt.ylim(0, 35)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Save to a file
filepath = '/mnt/data/monthly_bar_chart.png'
plt.savefig(filepath, bbox_inches='tight')
plt.close()

filepath
The colorful bar chart showing the monthly data has been created and saved. You can download it using the link below:

[Download monthly_bar_chart.png](sandbox:/mnt/data/monthly_bar_chart.png)

✅ Agent response complete
The colorful bar chart showing the monthly data has been created and saved. You can download it using the link below:

[Download monthly_bar_chart.png](sandbox:/mnt/data/monthly_bar_ch

## 5. Collect File IDs from Chat Response

In [10]:
# Extract file IDs from the response items
file_ids = []

# Check for any items with file_id attribute
for item in response.items:
    if hasattr(item, 'file_id') and item.file_id:
        file_ids.append(item.file_id)
        print(f"📁 Found file ID in item: {item.file_id}")

# Alternative: Look for FileReferenceContent in the response items
if not file_ids:
    for item in response.items:
        if hasattr(item, 'content_type') and item.content_type == 'file_reference':
            if hasattr(item, 'file_id') and item.file_id:
                file_ids.append(item.file_id)
                print(f"📁 Found file reference: {item.file_id}")


print(f"📋 Collected file IDs: {file_ids}")
print(f"📄 Total files to download: {len(file_ids)}")

# Optional: Fallback to thread messages if no files found in response
if not file_ids:
    print("⚠️ No files found in response. Checking thread messages as backup...")
    messages = foundry_client.messages.list(thread_id=thread.id, order="asc")
    
    for msg in messages.data:
        if msg.role == "assistant":
            # Check for image files and annotations
            if hasattr(msg, 'image_contents'):
                for img in msg.image_contents:
                    if hasattr(img, 'image_file') and hasattr(img.image_file, 'file_id'):
                        file_id = img.image_file.file_id
                        if file_id not in file_ids:
                            file_ids.append(file_id)
                            print(f"🖼️ Found image file: {file_id}")
            
            if hasattr(msg, 'file_path_annotations'):
                for ann in msg.file_path_annotations:
                    if hasattr(ann, 'file_path') and hasattr(ann.file_path, 'file_id'):
                        file_id = ann.file_path.file_id
                        if file_id not in file_ids:
                            file_ids.append(file_id)
                            print(f"📄 Found file annotation: {file_id}")
    
    print(f"📄 Total files found: {len(file_ids)}")

📁 Found file ID in item: assistant-EQXoouisMr1mMrq8UENLkg
📋 Collected file IDs: ['assistant-EQXoouisMr1mMrq8UENLkg']
📄 Total files to download: 1


## 6. Retrieve File Contents & Save Locally

In [12]:
# Retrieve file contents and save locally using Microsoft's recommended pattern
from azure.ai.projects import AIProjectClient

# Create project client for file operations (Microsoft's recommended approach)
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=credential
)

with project_client:
    for i, file_id in enumerate(file_ids, 1):
        try:
            print(f"📁 Retrieving content for file {i}: {file_id}")
            
            # Use Microsoft's recommended pattern: project_client.agents.files
            file_content_iterator = project_client.agents.files.get_content(file_id)
            
            # Collect all byte chunks from the iterator
            file_data = b''.join(chunk for chunk in file_content_iterator)
            
            # Save content to local file
            filename = f"chart_{i}_{file_id[:8]}.png"
            file_path = downloads_dir / filename
            
            with open(file_path, 'wb') as f:
                f.write(file_data)
            
            print(f"✅ Downloaded: {file_path}")
            
        except Exception as e:
            print(f"❌ Error retrieving {file_id}: {e}")

print(f"\n🎉 Files downloaded to: {downloads_dir}")

📁 Retrieving content for file 1: assistant-EQXoouisMr1mMrq8UENLkg
✅ Downloaded: c:\Users\chrismckee\Documents\GitHub\SKNotebooks\azure_ai_files\downloads\chart_1_assistan.png

🎉 Files downloaded to: c:\Users\chrismckee\Documents\GitHub\SKNotebooks\azure_ai_files\downloads
✅ Downloaded: c:\Users\chrismckee\Documents\GitHub\SKNotebooks\azure_ai_files\downloads\chart_1_assistan.png

🎉 Files downloaded to: c:\Users\chrismckee\Documents\GitHub\SKNotebooks\azure_ai_files\downloads


## 7. Cleanup Resources

In [14]:
# Clean up Azure resources with proper None checks
try:
    # Delete thread if it exists
    if thread is not None:
        await thread.delete()
        print("✅ Thread deleted")
    else:
        print("ℹ️ Thread already cleaned up")
        
    # Delete agent if foundry_client and agent_definition exist
    if foundry_client is not None and agent_definition is not None:
        foundry_client.delete_agent(agent_definition.id)
        print("✅ Agent deleted")
    else:
        print("ℹ️ Agent already cleaned up")
        
    # Close Semantic Kernel client if it exists
    if sk_client is not None:
        await sk_client.__aexit__(None, None, None)
        print("✅ SK client closed")
    else:
        print("ℹ️ SK client already cleaned up")
        
    # Close project client if it exists (from file operations)
    if 'project_client' in locals() and project_client is not None:
        project_client.close()
        print("✅ Project client closed")
        
except Exception as e:
    print(f"⚠️ Cleanup error (continuing): {e}")

# Reset variables to None
thread = None
agent = None
sk_client = None
foundry_client = None
agent_definition = None
file_ids = None

print("✅ All resources cleaned up!")
print("💡 Downloaded files preserved locally")

ℹ️ Thread already cleaned up
ℹ️ Agent already cleaned up
ℹ️ SK client already cleaned up
✅ Project client closed
✅ All resources cleaned up!
💡 Downloaded files preserved locally
